# Equality Constrained Entropy Maximization


This is an exercise from [Convex Optimization](https://web.stanford.edu/~boyd/cvxbook/)

$$ \text{minimize } \qquad f(x) = \displaystyle\sum_{i=1}^n x_i\log(x_i) $$

$$ \text{subject to } \qquad Ax=b $$

## Feasible Start

For now, we assume that we have a feasible point $x$, so we assume that we have a point $x$ such that $Ax=b$.

Let's replace the objective with its second order Taylor Approximation at $x$.

$$ \hat{f}(v) = f(x) + \nabla f(x)^T (v-x) + \frac{1}{2} (v-x)^T \nabla^2 f(x) (v-x) $$

$$ \text{minimize } \qquad \hat{f}(x+v) = \frac{1}{2} v^T \nabla^2 f(x) v + \nabla f(x)^Tv + f(x)$$

$$ \text{subject to} \qquad A(x+v)=b$$

with respect to the variable $v$. We can solve this problem and we think of the solution as the value which needs to be added to $x$ to minimize the quadratic approximation. This value is the Newton step.

We can solve the quadratic approximation minimization problem (and therefore find the Newton step) by forming the dual and solving an unconstrained minimization problem.

$$ \text{minimize } \qquad \frac{1}{2} v^T \nabla^2 f(x) v + \nabla f(x)^T v + f(x) + \lambda^T(A(x+v)-b) $$

A solution to this problem is equivalent to finding a solution $v^{\star}, \lambda^{\star}$ to the KKT equations

$$A(x+v^{\star})=b \qquad \text{ and } \qquad \nabla^2 f(x)v^{\star} + \nabla f(x) + A^T \lambda^{\star} = 0$$

Note that we compute the gradients with respect to the variable $v$. 

We re-write the equations

$$Av^{\star}=b-Ax = 0 \qquad \text{ and } \qquad \nabla^2 f(x)v^{\star} +  A^T \lambda^{\star} = -\nabla f(x)$$


We can compute the Newton step by solving the KKT system

$$ \begin{bmatrix} \nabla^2f(x) & A^T \\ A & 0 \end{bmatrix} \begin{bmatrix} v^{\star} \\ \lambda^{\star} \end{bmatrix}  = \begin{bmatrix} -\nabla f(x) \\ 0 \end{bmatrix}$$

where $\Delta x_{nt} = v^{\star}$ is the Newton step (the step we use to update our current feasible point) and $w = \lambda^{\star}$ is the updated dual variable (already updated, no step necessary).

In [49]:
import numpy as np
import cvxpy as cp
import plotly.graph_objects as go
import scipy

In [50]:
# generate problem data
n = 100
p = 30
A = np.random.normal(0,1,(p,n))
x_hat = np.random.uniform(0,10,100) # use this as feasible intial point
b = A@x_hat 

In [51]:
# solve KKT system via block elimination
def KKT_solve(H,A,g,h):
    H_inv = np.linalg.inv(H)
    X = H_inv@A.T
    y = H_inv@g
    S_inv = np.linalg.inv(-A@X)
    w = S_inv@(A@y-h)
    return H_inv@(-A.T@w-g), w
def l2_norm(x):
    return np.sqrt(np.sum(x**2))
def entropy(x):
    return np.sum(x*np.log(x))
def grad(x):
    return np.log(x)+1
def hessian(x):
    return np.diag(1/x)
def backtrack(x, objective, alpha, beta, grad, descent_direction):
    t = 1
    while np.min(x+t*descent_direction)<=0: # check that updated point will be in the domain
        t = t*beta
    while objective(x+t*descent_direction) > objective(x) + alpha*t*(grad@descent_direction):
        t = t*beta
    return t

In [52]:
def Newtons_method(obj, A, b, x_init, grad_func, hessian_func, tol=1e-8, max_iter=1000, alpha=0.1, beta=0.5 ):
    x = x_init
    i = 0
    if (A@x-b == 0).all():
        print('Feasible start')
        while i <= max_iter:
            g = grad_func(x)
            H = hessian_func(x)
            delta_x, w = KKT_solve(H,A,g,np.zeros(A.shape[0]))
            dec = delta_x@(H@delta_x)
            if dec < tol:
                break
            else:
                t = backtrack(x, obj, alpha, beta, g, delta_x)
                x = x+t*delta_x
            i += 1
    else:
        print('Infeasible Start')
        w = np.zeros(A.shape[0])
        while i <= max_iter:
            g = grad_func(x)
            H = hessian_func(x)
            delta_x, updated_w = KKT_solve(H,A,g,A@x-b)
            delta_w = updated_w - w
            t = r_backtrack(x,w,delta_x,delta_w,A,b, alpha, beta)
            x = x+t*delta_x
            w = w+t*delta_w
            if l2_norm(A@x-b) < tol and l2_norm(r(x,w,A,b))< tol:
                break
            i += 1
            
    return x

In [53]:
# feasible start example
solution = Newtons_method(entropy, A,b, x_hat, grad, hessian)
entropy(solution)

Feasible start


218.99175125160755

## Infeasible Start

Previously, we have seen how to find the Newton step by solving the KKT equations

$$Av^{\star}=b-Ax = 0 \qquad \text{ and } \qquad \nabla^2 f(x)v^{\star} +  A^T \lambda^{\star} = -\nabla f(x)$$

Note that if the point $x$ is infeasible, $Ax-b \neq 0$. So the KKT system becomes

$$ \begin{bmatrix} \nabla^2f(x) & A^T \\ A & 0 \end{bmatrix} \begin{bmatrix} v^{\star} \\ \lambda^{\star} \end{bmatrix}  = -\begin{bmatrix} \nabla f(x) \\ Ax-b \end{bmatrix}$$

We want to update the dual variable as well to approximately satisfy the optimality conditions. From the KKT conditions for the original problem, we want 

$$ r_{\text{dual}} (x, \lambda) = \nabla f(x) + A^T \lambda \qquad \qquad r_{\text{primal}}(x, \lambda) = Ax-b$$

the dual and primal residuals to be 0. So, we want to update the primal and dual variables to drive the entries of 

$$ r(x ,\lambda) = (r_{\text{dual}} (x, \lambda),r_{\text{primal}}(x, \lambda)) $$

to 0. 

In [54]:
def r(x,w,A,b):
    return np.append(grad(x)+A.T@w, A@x-b)
def r_backtrack(x,w,delta_x,delta_w,A,b, alpha, beta):
    t = 1
    l2 = l2_norm(r(x,w,A,b))
    while np.min(x+t*delta_x) <= 0: # the gradient involves log, need to have updated point in domain
        t = t*beta
    while l2_norm(r(x+t*delta_x, w+t*delta_w,A,b)) > (1-alpha*t)*l2:
        t = beta*t
    return t

In [55]:
entropy(Newtons_method(entropy, A, b, np.ones(A.shape[1]),grad, hessian))

Infeasible Start


218.99175125160863

In [56]:
x = cp.Variable(n)
objective = cp.Minimize(cp.sum(-1*cp.entr(x)))
constraints = [A@x==b]
prob = cp.Problem(objective, constraints)
prob.solve()

218.991750660826

## Solve the dual of the original problem

Original Problem

$$ \text{minimize } \qquad f(x) = \displaystyle\sum_{i=1}^n x_i\log(x_i) $$

$$ \text{subject to } \qquad Ax=b $$

Dual Problem

$$ \text{maximize } \qquad g(\lambda) = \text{inf} \left( f(x) + \lambda^T(Ax-b)\right)$$

We can express the dual function $g$ using the conjugate $f^{*}$ of the objective function $f$:

$$g(\lambda) = -b^T\lambda - f^{*} (-A^T \lambda)$$

The conjugate of negative entropy is 

$$ f^{*} (y) = \displaystyle\sum_{i=1}^n \text{exp}(y_i-1) $$

Our goal is to solve the problem

$$ \text{maximize} \qquad -b^T\lambda - \displaystyle\sum_{i=1}^n \text{exp}(-a_i^T\lambda - 1)$$

where $a_i$ is the $i$th column of the matrix $A$. We solve the equivalent problem

$$ \text{minimize} \qquad b^T\lambda + \displaystyle\sum_{i=1}^n \text{exp}(-a_i^T\lambda - 1)$$

with respect to the variable $\lambda$.

In [73]:
def dual_backtrack(x, objective,A, b, alpha, beta, gradient, descent_direction):
    t = 1
    while objective(x+t*descent_direction, A, b) > objective(x,A,b)+alpha*t*gradient@descent_direction:
        t = t*beta
    return t
def dual_grad(x,A,b):
    return b-A@(np.exp(-A.T@x-1))
def dual_hessian(x, A, b):
    return A@np.diag(np.exp(-A.T@x-1))@A.T
def dual_objective(x,A,b):
    return b@x + np.sum(np.exp(-A.T@x-1))

In [80]:
# Newton's Method
z = np.zeros(A.shape[0])
tol = 1e-8 
max_iter = 100
alpha = 0.1 # line search
beta = 0.5 # line search
i = 0
while i <= max_iter:
    g = dual_grad(z, A, b)
    H = dual_hessian(z, A, b)
    newton_step = -np.linalg.inv(H)@g
    dec = -g@newton_step
    if dec < tol:
        break
    else:
        t = dual_backtrack(z, dual_objective,A,b,alpha, beta, g, newton_step)
        z = z+t*newton_step
    i +=1

In [85]:
-dual_objective(z,A,b)

218.9917512515649

Suppose we find a solution $\lambda^*$ to the dual problem. Then, if we can find a point $x^*$ which minimizes the Lagrangian $L(x,\lambda^*)$, then $x^*$ is a primal optimal point. 

$$L(x, \lambda^*) = \displaystyle\sum_{i=1}^n x_i\log(x_i) + A^T \lambda^* $$

We can compute the gradient, set its entries equal to 0, and solve to find an optimal primal point.

$$ [\nabla L(x, \lambda^*)]_i = \log(x_i) + 1 + a_i^T\lambda^* = 0 $$

$$ x_i = \text{exp}(-a_i^T-1) $$

In [86]:
np.exp(-A.T@z-1)

array([ 1.34334804,  0.09825093,  1.41879333,  2.09632328,  2.12707902,
        1.79612066,  3.04164402,  1.0391971 ,  1.58946936,  0.02585689,
        0.33200857,  0.65157311,  0.40973208,  0.44675065,  4.04244539,
        2.6315314 ,  0.05165718,  1.27951011,  7.45419904,  1.24650019,
        0.20762563,  0.91567377,  3.12948152,  3.11113851,  4.93902936,
        1.44618547, 12.77851283,  3.52911626, 11.02331712,  2.61983735,
        4.02838263,  0.16815896,  0.59512935,  0.52279157,  0.4849513 ,
        0.65701703,  2.14008806,  7.47662646,  1.38770218,  0.1382485 ,
        0.05580886,  0.30922579,  0.04741908,  0.51898547,  0.92303269,
        0.54110112,  0.27637043,  1.1709028 ,  3.63364308,  1.41175495,
        6.20678069,  0.83365602,  0.02542982,  1.89561086,  4.5765728 ,
        3.01566282,  0.36505276,  0.05132818,  0.81037054,  2.35607558,
        2.05845361,  2.12914127,  0.08766997,  1.15716089,  0.03357572,
        1.35149981,  0.26762997,  3.70224108,  0.18464198,  0.80

In [76]:
z = cp.Variable(30)
objective = cp.Maximize(-b@z-cp.sum(cp.exp(-A.T@z-1)))
prob = cp.Problem(objective,[])
prob.solve()

218.9917510533116

In [78]:
z.value

array([ 0.34340259,  0.10697478, -0.25283001,  0.21451717, -0.51255088,
        0.16653594,  0.61571971, -0.29855132, -0.11562343, -0.14247313,
        0.24427931,  0.41259437, -0.05106605, -0.0946379 ,  0.23959577,
        0.60849946, -0.57510286,  0.27093009,  0.27722523,  0.19888811,
       -0.24309424,  0.07629719, -0.77717196, -0.73595885,  0.15500217,
        0.4537255 , -0.25244459,  0.00338   ,  0.03396786,  0.31713797])